In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy import stats
import seaborn as sns
from sklearn.metrics import mean_squared_error

# Carga de los df de features para entrenar y predecir

In [9]:
features_18_20 = pd.read_csv("windows/18_20/features.csv") 
features_19_21 = pd.read_csv("windows/19_21/features.csv")
features_20_22 = pd.read_csv("windows/20_22/features.csv")
features_21_23 = pd.read_csv("windows/21_23/features.csv")
features_24_26 = pd.read_csv("windows/24_26/features.csv") 

In [2]:
label_auc_18_20 = pd.read_csv("windows/18_20/labels_auc.csv")
label_auc_19_21 = pd.read_csv("windows/19_21/labels_auc.csv")
label_auc_20_22 = pd.read_csv("windows/20_22/labels_auc.csv")
label_auc_21_23 = pd.read_csv("windows/21_23/labels_auc.csv")

In [3]:
label_inst_18_20 = pd.read_csv("windows/18_20/labels_inst.csv")
label_inst_19_21 = pd.read_csv("windows/19_21/labels_inst.csv")
label_inst_20_22 = pd.read_csv("windows/20_22/labels_inst.csv")
label_inst_21_23 = pd.read_csv("windows/21_23/labels_inst.csv")

Clasificador

In [71]:
#labels_clas_auc_18_20 = pd.DataFrame({'ref_hash': label_auc_18_20['ref_hash'], 'label_auc': label_auc_18_20['label_auc'] == 259200})
#labels_clas_auc_19_21 = pd.DataFrame({'ref_hash': label_auc_19_21['ref_hash'], 'label_auc': label_auc_19_21['label_auc'] == 259200})
#labels_clas_auc_20_22 = pd.DataFrame({'ref_hash': label_auc_20_22['ref_hash'], 'label_auc': label_auc_20_22['label_auc'] == 259200})
#labels_clas_auc_21_23 = pd.DataFrame({'ref_hash': label_auc_21_23['ref_hash'], 'label_auc': label_auc_21_23['label_auc'] == 259200})

In [ ]:
#labels_clas_inst_18_20 = pd.DataFrame({'ref_hash': label_inst_18_20['ref_hash'], 'label_inst': label_inst_18_20['label_inst'] == 259200})
#labels_clas_inst_19_21 = pd.DataFrame({'ref_hash': label_inst_19_21['ref_hash'], 'label_inst': label_inst_19_21['label_inst'] == 259200})
#labels_clas_inst_20_22 = pd.DataFrame({'ref_hash': label_inst_20_22['ref_hash'], 'label_inst': label_inst_20_22['label_inst'] == 259200})
#labels_clas_inst_21_23 = pd.DataFrame({'ref_hash': label_inst_21_23['ref_hash'], 'label_inst': label_inst_21_23['label_inst'] == 259200})

# Funciones de entrenamiento

In [ ]:
def guardar_submit(params, result):
    tiempo = "time"
    with open("historial_submits.txt","a+") as f:
        f.write("\n"+tiempo+"|"+params+"|")

In [ ]:
def entrenar_modelo(params):
    result = None#entrenar(params)
    guardar_submit(params, result)

In [5]:
def modelfit(model, features, labels, cv_folds=5, early_stopping_rounds=50):
    parametros = model.get_xgb_params()
    dmatrix = xgb.DMatrix(features.values, label=labels.values)
    cvresult = xgb.cv(parametros, dmatrix, num_boost_round=model.get_params()['n_estimators'],
                      nfold=cv_folds,metrics='rmse', #verbose_eval=True,
                      early_stopping_rounds=early_stopping_rounds)
    model.set_params(n_estimators=cvresult.shape[0])
    model.fit(features, labels,eval_metric='rmse')

In [6]:
def entrenar(modelo, df_features, labels):
    df_features = df_features.merge(labels, how="left", left_on="ref_hash", right_on="ref_hash")
    df_features.set_index("ref_hash", inplace=True)
    X, y = df_features.iloc[:,:-1], df_features.iloc[:,-1:]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

    modelo.fit(X_train, y_train, eval_metric='rmse')
    #modelfit(modelo, X_train, y_train, early_stopping_rounds=30)

    prediction = modelo.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, prediction))
    print("RMSE: %f" % (rmse))
    return prediction

# Entrenamiento

In [25]:
model_auc = xgb.XGBRegressor(
    #booster='dart',
    learning_rate =0.01,
    n_estimators=1000,
    max_depth=4,
    min_child_weight=8,
    gamma=0.22,
    subsample=1,
    colsample_bytree=1,
    objective='reg:squarederror',
    nthread=-1,
    scale_pos_weight=18.8,
    #sample_type='weighted',
    #rate_drop=0.1,
    #skip_dropout=0.5,
    random_state=272
)

In [26]:
entrenar(model_auc, features_18_20, label_auc_18_20)
entrenar(model_auc, features_19_21, label_auc_19_21)
entrenar(model_auc, features_20_22, label_auc_20_22)
entrenar(model_auc, features_21_23, label_auc_21_23)

RMSE: 84310.805713
RMSE: 86582.514091
RMSE: 84881.304576
RMSE: 83655.051073


array([166529.88 , 232779.66 , 232611.44 , ...,  25129.111,  84525.56 ,
       232779.66 ], dtype=float32)

In [27]:
model_inst = xgb.XGBRegressor(
    #booster='dart',
    learning_rate =0.01,
    n_estimators=1000,
    max_depth=4,
    min_child_weight=8,
    gamma=0.22,
    subsample=1,
    colsample_bytree=1,
    objective='reg:squarederror',
    nthread=-1,
    scale_pos_weight=18.8,
    #sample_type='weighted',
    #rate_drop=0.1,
    #skip_dropout=0.5,
    random_state=272
)

In [28]:
entrenar(model_inst, features_18_20, label_inst_18_20)
entrenar(model_inst, features_19_21, label_inst_19_21)
entrenar(model_inst, features_20_22, label_inst_20_22)
entrenar(model_inst, features_21_23, label_inst_21_23)

RMSE: 52184.751285
RMSE: 51852.940499
RMSE: 51164.105994
RMSE: 50584.160513


array([235279.12, 256176.05, 246305.78, ..., 233702.88, 233475.81,
       256176.05], dtype=float32)

## Prueba filtrando los tpos máximos (las predicciones, al parecer, mejoran)

In [9]:
label_inst_test = label_inst_21_23.loc[label_inst_21_23['label_inst'] != 259200].set_index('ref_hash')

In [10]:
features_inst_test = features_21_23.set_index('ref_hash').join(label_inst_test, how = 'inner')
features_inst_test.drop(columns = ["label_inst"], inplace = True)

In [11]:
label_inst_test.reset_index(inplace = True)
features_inst_test.reset_index(inplace= True)

In [12]:
entrenar(model_inst, features_inst_test, label_inst_test)

RMSE: 76970.157667


array([135006.81, 131146.22, 123459.47, ..., 144312.05, 130768.53,
       133122.8 ], dtype=float32)

In [13]:
label_auc_test = label_auc_21_23.loc[label_auc_21_23['label_auc'] != 259200].set_index('ref_hash')

In [14]:
features_auc_test = features_21_23.set_index('ref_hash').join(label_auc_test, how = 'inner')
features_auc_test.drop(columns = ["label_auc"], inplace = True)

In [15]:
label_auc_test.reset_index(inplace = True)
features_auc_test.reset_index(inplace= True)

In [16]:
entrenar(model_auc, features_auc_test, label_auc_test)

RMSE: 64394.910498


array([ 50061.867 ,   4167.8184, 113379.02  , ...,  79042.24  ,
        87008.75  , 104195.27  ], dtype=float32)

# Predecir

In [29]:
pred_auctions = model_auc.predict(features_24_26.set_index("ref_hash"))
df_preds_auctions = pd.DataFrame({'ref_hash' : features_24_26['ref_hash'], 'obj' : pred_auctions})

In [30]:
pred_installs = model_inst.predict(features_24_26.set_index("ref_hash"))
df_preds_installs = pd.DataFrame({'ref_hash' : features_24_26['ref_hash'], 'obj' : pred_installs})

# Feature Importance

In [ ]:
from sklearn.ensemble import RandomForestRegressor

sns.set(rc={'figure.figsize':(10,10)})
names = .columns
rf = RandomForestRegressor(
    random_state=123,
    n_estimators=400,
    min_samples_split=3
)
rf.fit(X_train.values, y_train.values.ravel())
impor = np.vstack((names, rf.feature_importances_)).T
imporDf = pd.DataFrame(impor)
imporDf.columns = ['feature', 'importancia']
imporDf = imporDf.set_index('feature').sort_values(by=['importancia'], ascending=False).iloc[::-1]
ax = imporDf.plot(kind='barh')
ax.set_title("Importancia de cada feature segun Random Forests")
ax.set(xlabel="Importancia", ylabel="Feature")
plt.figure(figsize=(10,10))

# Submit to Kaggle

In [19]:
def export_df(df, name):
    df.to_csv(name, index=False)

Las predicciones tendrán seteadas como índice los ref_hash para no perder la referencia
No es necesario filtrar los ref_hash y quedarnos solo con los target en las predicciones que obtenemos ya que de eso
se encarga la función 

In [20]:
target = pd.read_csv("target_competencia_ids.csv")

In [21]:
def create_submit_df(auctions_predictions, installs_predictions, target):
    
    target = target.set_index('ref_hash')
    
    auc = auctions_predictions.reset_index()
    auc.columns = ['ref_hash','obj']
    auc['ref_hash'] = auc['ref_hash'].astype(str) + "_sc"
    auc = auc.set_index('ref_hash')
    
    ins = installs_predictions.reset_index()
    ins.columns = ['ref_hash','obj']
    ins['ref_hash'] = ins['ref_hash'].astype(str) + "_st"
    ins = ins.set_index('ref_hash')
    
    frames = [ins,auc]
    submit_result = pd.concat(frames).reset_index()
    target_list = target.reset_index('ref_hash')['ref_hash'].tolist()
    return submit_result.loc[submit_result['ref_hash'].isin(target_list)].sort_values(by = 'ref_hash')

In [31]:
kaggle_sub = create_submit_df(df_preds_auctions.set_index('ref_hash'), df_preds_installs.set_index('ref_hash'), target)

In [23]:
len(kaggle_sub)

8074

In [32]:
export_df(kaggle_sub, "submit.csv")